In [1]:
import os
import numpy as np
import tensorflow as tf


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,load_img, img_to_array
import random

dataGen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

#Identify folder path
folder_path = 'BirdImages/Sparrow/'
augmented_folder = 'BirdImages/Sparrow/Augmented/'

os.makedirs(augmented_folder, exist_ok=True)

#Listing images labeled with 'jpg' in the sparrow folder
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

#Randomly selects images for augmentation
selected_images = random.sample(image_files, 20)

augmented_images_count=0

for image_file in selected_images:
    img_path = os.path.join(folder_path,image_file)
    img = load_img(img_path)
    img = img_to_array(img) 
    img = img.reshape((1,) + img.shape)
    augmented_images = []
    for batch in dataGen.flow(img, batch_size=1, save_to_dir=augmented_folder, save_format='jpg'):
        augmented_images.append(batch[0].astype('uint8'))
        augmented_images_count +=1
        if len(augmented_images) >=200:
            break
    if augmented_images_count>=200:
        break

print(f"Generated {augmented_images_count} augmented images.")

Generated 200 augmented images.


In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'BirdImages/',
    validation_split = 0.2,
    subset = "training",
    seed = 1337,
    image_size=(128,128),
    batch_size=32
)
train_ds_augmented = tf.keras.preprocessing.image_dataset_from_directory(
    'BirdImages/Augmented',
    subset = "training",
    image_size=(128,128),
    batch_size=32
)

combined_train_ds = train_ds.concatenate(train_ds_augmented)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'BirdImages/',
    validation_split = 0.2,
    subset = "validation",
    seed = 1337,
    image_size=(128,128),
    batch_size=32
)

: 

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32,(3,3), activation = 'relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(5 , activation='softmax')
])

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 64)       0

In [4]:
history = model.fit(
    train_ds,  # Training dataset
    validation_data=val_ds,  # Validation dataset
    epochs=25,  # Number of epochs (you can adjust this)
    batch_size=32  # Batch size (this is already set in the dataset loader)
)

Epoch 1/25
156/156 [==============================] - 47s 290ms/step - loss: 5.2632 - accuracy: 0.3113 - val_loss: 1.4328 - val_accuracy: 0.3599
Epoch 2/25
156/156 [==============================] - 38s 243ms/step - loss: 1.4290 - accuracy: 0.3652 - val_loss: 1.4354 - val_accuracy: 0.3680
Epoch 3/25
156/156 [==============================] - 42s 271ms/step - loss: 1.3918 - accuracy: 0.3922 - val_loss: 1.4008 - val_accuracy: 0.3736
Epoch 4/25
156/156 [==============================] - 47s 304ms/step - loss: 1.3364 - accuracy: 0.4233 - val_loss: 1.4114 - val_accuracy: 0.3994
Epoch 5/25
156/156 [==============================] - 59s 380ms/step - loss: 1.2614 - accuracy: 0.4575 - val_loss: 1.4150 - val_accuracy: 0.3857
Epoch 6/25
156/156 [==============================] - 60s 383ms/step - loss: 1.2267 - accuracy: 0.4771 - val_loss: 1.4578 - val_accuracy: 0.3921
Epoch 7/25
156/156 [==============================] - 69s 440ms/step - loss: 1.1474 - accuracy: 0.4986 - val_loss: 1.4550 - val_ac

In [5]:
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

39/39 [==============================] - 3s 66ms/step - loss: 1.4842 - accuracy: 0.7166
Validation Accuracy: 71.66%
